<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Prediction/ProtBert_BFD_Predict_MS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>Stability Score prediction using ProtTrans model</h3>

**1. Load necessry libraries including huggingface transformers**

In [ ]:
!pip install -q transformers

In [ ]:
!pip install tape-proteins

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
import re
from typing import Union, List, Tuple, Sequence, Dict, Any, Optional
from copy import copy
from pathlib import Path
import pickle as pkl
import logging
import random
import tqdm
import numpy as np
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import spearmanr
import torch
import torch.nn as nn
import lmdb
from tape import ProteinBertModel, TAPETokenizer
from transformers import T5Tokenizer, T5EncoderModel
import torch.nn.functional as F
from torch.utils.data import Dataset
from scipy.spatial.distance import pdist, squareform

**2. Load the sequence classification pipeline and load it into the GPU if avilabile**

In [ ]:
#run with gpu
pipeline = TextClassificationPipeline(
    model=AutoModelForSequenceClassification.from_pretrained("Rostlab/prot_bert_bfd_membrane"),
    tokenizer=AutoTokenizer.from_pretrained("Rostlab/prot_bert_bfd_membrane"),
    device=0
)

**3. Load sequences and labels from TAPE**

In [ ]:
logger = logging.getLogger(__name__)

class LMDBDataset(Dataset):
    """Creates a dataset from an lmdb file.
    Args:
        data_file (Union[str, Path]): Path to lmdb file.
        in_memory (bool, optional): Whether to load the full dataset into memory.
            Default: False.
    """

    def __init__(self,
                 data_file: Union[str, Path],
                 in_memory: bool = False):

        data_file = Path(data_file)
        if not data_file.exists():
            raise FileNotFoundError(data_file)

        env = lmdb.open(str(data_file), max_readers=1, readonly=True,
                        lock=False, readahead=False, meminit=False)

        with env.begin(write=False) as txn:
            num_examples = pkl.loads(txn.get(b'num_examples'))

        if in_memory:
            cache = [None] * num_examples
            self._cache = cache

        self._env = env
        self._in_memory = in_memory
        self._num_examples = num_examples

    def __len__(self) -> int:
        return self._num_examples

    def __getitem__(self, index: int):
        if not 0 <= index < self._num_examples:
            raise IndexError(index)

        if self._in_memory and self._cache[index] is not None:
            item = self._cache[index]
        else:
            with self._env.begin(write=False) as txn:
                item = pkl.loads(txn.get(str(index).encode()))
                if 'id' not in item:
                    item['id'] = str(index)
                if self._in_memory:
                    self._cache[index] = item
        return item

In [ ]:
train_data = LMDBDataset('stability_train.lmdb')
test_data = LMDBDataset('stability_test.lmdb') 
print(train_data.__getitem__(900))

{'id': b'EEHEE_rd1_0307.pdb_hp', 'primary': 'GSWKGYATANKKQAPTEEYLKDNAEQSGVDYAEKTKGKLEVDK', 'protein_length': 43, 'topology': b'EEHEE', 'parent': b'EEHEE_rd1_0307', 'stability_score': array([-1.], dtype=float32)}


In [ ]:
train_sequences=[]
train_stability_scores=[]
for i in range(len(train_data)):
    train_sequences.append(train_data.__getitem__(i)['primary'])
    train_stability_scores.append(train_data.__getitem__(i)['stability_score'][0])
print(train_sequences)
print(train_stability_scores)

['...']
[0.17, -0.18, 1.36, 0.65, -0.47, -0.15, 0.06, -0.9, -0.77, -0.32, 0.4, 0.03, -1.18, -0.65, -0.42, 0.26, -0.24, -0.39, 0.2, -0.66, 0.17, -0.32, -0.65, -0.12, 0.01, -0.5, -0.13, -0.29, -0.15, -0.31, 0.06, 0.21, -0.07, 0.01, -0.02, 0.04, -0.04, 0.14, 0.19, 0.15, 0.23, 0.11, -0.05, 0.39, -1.09, 0.3, 0.28, 0.15, 0.06, 0.31, -0.1, 0.33, 0.3, 0.01, 0.18, 0.24, -0.29, -0.16, -0.13, -0.34, -0.09, 0.1, 0.31, 0.44, -0.17, -0.54, 0.11, 0.06, 0.19, -0.0, 0.15, -0.55, -0.14, -0.18, 0.13, 0.72, 0.04, -0.27, 1.28, -0.02, 0.46, 0.25, 0.12, 0.03, -1.0, -0.12, 0.43, -0.35, 0.33, -0.55, -0.09, 0.67, 0.06, -0.15, -0.08, -0.46, -0.4, 1.01, -0.72, -0.03, -0.13, 0.15, -0.16, 0.85, -0.05, 0.12, 0.55, 0.27, 0.87, 0.04, -0.04, 1.01, -0.24, 0.29, 0.44, -0.32, -0.06, -0.33, 0.11, 0.82, -0.13, 0.1, -0.25, 0.69, -0.05, 0.54, -0.27, 0.27, 0.17, 1.03, -0.32, -0.1, 0.56, 0.07, -0.3, -0.45, -0.25, 1.08, -0.18, -0.21, 0.04, 0.53, 0.29, -0.21, 0.6, -0.53, -0.94, -0.14, 0.27, -0.18, -0.18, -0.81, -0.63, 0.4, 0.15, 

In [ ]:
test_sequences=[]
test_stability_scores=[]
for i in range(len(test_data)):
    test_sequences.append(test_data.__getitem__(i)['primary'])
    test_stability_scores.append(test_data.__getitem__(i)['stability_score'][0])
print(test_sequences)
print(test_stability_scores)

['...']
[0.85, 0.3, 0.23, 0.53, 0.55, 0.26, 0.62, 0.2, 0.59, 0.68, 0.32, 0.29, 0.27, 0.17, 0.56, 0.6, 0.86, 0.55, 0.31, 0.25, 0.63, 1.08, 0.63, 0.57, 0.66, 0.75, 0.86, 0.85, 0.35, 0.36, 0.63, 0.7, 0.57, 0.49, 0.77, 1.05, 1.1, 0.39, 0.51, 1.48, 0.6, 0.82, 0.93, 0.92, 1.31, 0.92, 0.58, 0.02, 0.83, 1.08, 0.42, 0.36, 0.92, 1.56, 1.41, -0.08, 0.22, 1.23, 0.16, 0.38, 0.92, 0.43, 1.25, 1.08, 0.19, 0.03, 0.33, 0.53, 0.28, 0.19, 0.91, 1.03, 0.93, 0.81, 0.87, 0.79, 0.85, 0.82, 0.65, 0.87, 0.78, 0.94, 0.92, 0.47, 0.84, 0.95, 0.71, 0.72, 0.69, 0.8, 0.76, 0.77, 0.8, 0.79, 0.68, 0.71, 0.85, 0.72, 0.78, 0.81, 0.72, 0.66, 0.81, 0.79, 0.68, 0.68, 0.77, 0.87, 0.81, 0.77, 0.9, 0.91, 0.77, 0.82, 0.89, 0.77, 0.87, 0.82, 0.84, 0.43, 0.86, 0.84, 0.79, 0.78, 0.88, 0.92, 0.8, 0.84, 0.83, 0.68, 0.67, 0.76, 0.83, 0.63, 0.72, 0.9, 0.79, 0.58, 0.81, 0.71, 0.79, 0.81, 0.86, 0.73, 0.72, 0.65, 0.69, 0.74, 0.56, 0.65, 0.48, 0.65, 0.58, 0.62, 0.81, 0.36, 0.65, 0.64, 0.66, 0.53, 0.53, 0.38, 0.61, 0.68, 0.89, 0.69, 0.67,

**4. Load pre-trained model and make necessary modifications**

In [ ]:
# Load the pre-trained model and tokenizer
model_name = "Rostlab/prot_bert_bfd_membrane"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)
model.classifier = nn.Linear(model.classifier.in_features, 1)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define your training data and labels
train_texts = train_sequences  # a list of strings containing your training texts
train_labels = train_stability_scores  # a list of integers or labels corresponding to your training texts

# Define your test set and labels
test_texts = test_sequences  # a list of strings containing your test texts
test_labels = test_stability_scores  # a list of integers or labels corresponding to your test texts

train_labels = torch.tensor(train_labels).unsqueeze(-1)
test_labels = torch.tensor(test_labels).unsqueeze(-1)

# Convert your training and test texts to input tensors using the tokenizer
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
train_input_ids = torch.tensor(train_encodings['input_ids']).to(torch.int64)
test_input_ids = torch.tensor(test_encodings['input_ids']).to(torch.int64)
train_attention_mask = torch.tensor(train_encodings['attention_mask']).to(torch.int64)
test_attention_mask = torch.tensor(test_encodings['attention_mask']).to(torch.int64)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


**5. Train fine-tuned ProtTrans model (modified for stability score prediction)** 

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = nn.MSELoss()
train_dataset = torch.utils.data.TensorDataset(train_input_ids, train_attention_mask, train_labels)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

model.train()
for epoch in range(1):
    epoch_loss = 0.0
    progress_bar = tqdm.tqdm(train_dataloader, desc=f"Epoch {epoch + 1}", leave=False)
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels.float())
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        progress_bar.set_postfix({"batch_loss": loss.item()})
    avg_epoch_loss = epoch_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1} average loss: {avg_epoch_loss:.4f}")
    
torch.save(model.state_dict(), "model_weights.pth")

Epoch 1 average loss: 0.4767


**6. Use trained model to make predictions and compute accuracy metrics** 

In [ ]:
# Use the trained model to make predictions on the test set
with torch.no_grad():
    predictions = model(test_input_ids, attention_mask=test_attention_mask).logits.detach().cpu().numpy().flatten()

# Calculate and print R2 score
r2_score = r2_score(test_labels, predictions)

# Calculate and print RMSE
mse = mean_squared_error(test_labels, predictions)
rmse = np.sqrt(mse)

# Calculate and print Spearman's rho
rho, _ = spearmanr(test_labels, predictions)


In [ ]:
print("R2 score:", r2_score)
print("RMSE:", rmse)
print("Spearman's rho:", rho)

R2 score: -0.35
RMSE: 0.44
Spearman's rho: 0.71
